## **<center><font style="color:rgb(100,109,254)">Examination Answer Sheet Marks Entrance Using OCR</font> </center>**
### **<center><font style="color:rgb(100,109,254)">Machine Learning Project</font> </center>**

In [1]:
import cv2
print(cv2.__version__)
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"


4.6.0


In [2]:
import os
import numpy as np

imgq = cv2.imread("q1.jpeg")
h,w,c = imgq.shape
imgq = cv2.resize(imgq, (w//3, h//3))

orb = cv2.ORB_create(1000)
kp1, des1 = orb.detectAndCompute(imgq,None)
# imgKp1 = cv2.drawKeypoints(imgq, kp1, None)
per = 25

roi = [[(236, 294), (948, 388), 'text', 'StudentName'],
        [(216, 372), (692, 450), 'text', 'RegistrationNumber'],
        [(794, 382), (1254, 458), 'text', 'Subject'],
        [(1060, 448), (1260, 520), 'text', 'class'],
        [(1066, 646), (1250, 720), 'text', 'totalMarks'],
        [(1068, 864), (1256, 938), 'text', 'obtainedMarks']]

path = 'UserForms'
myImageList = os.listdir(path)
print(myImageList)

for j, y in enumerate(myImageList):
    img = cv2.imread(path+"/"+y)
    img = cv2.resize(img, (w//3, h//3))
    # img = cv2.resize(img, (w,h))
    # cv2.imshow("ss", img)
    kp2, des2 =  orb.detectAndCompute(img,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(des2, des1)
    # matches.sort(key= lambda x: x.distance)
    good = matches[:int(len(matches)*(per/100))]

    imgMatch = cv2.drawMatches(img, kp2, imgq, kp1, good[:100], None, flags=2)
    # imgMatch = cv2.resize(imgMatch, (w,h))
    # cv2.imshow(y, imgMatch)

    srcPonits = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dstPonits = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M, _ = cv2.findHomography(srcPonits, dstPonits, cv2.RANSAC, 5.0)
    imgScan = cv2.warpPerspective(img, M, (w//3, h//3))
    imgScan = cv2.resize(imgScan, (w,h))
    cv2.imshow(y, imgScan)

    imgShow = imgScan.copy()
    imgMask = np.zeros_like(imgScan)

    for x,r in enumerate(roi):

        cv2.rectangle(imgMask, (r[0][0], r[0][1]), (r[1][0], r[1][1]), (0,255,0), cv2.FILLED)
        imgShow = cv2.addWeighted(imgShow, 0.99, imgMask, 0.1, 0)

        imgCrop = imgScan[r[0][1]:r[1][1], r[0][0]:r[1][0]]
        cv2.imshow(str(x), imgCrop)

        if r[2] == 'text':
            confg = r'--oem 3 --psm 6 outputbase digits'
            print(f"{r[3]} : {pytesseract.image_to_string(imgCrop, config=confg)}")

    imgShow = cv2.resize(imgShow, (w,h))
    cv2.imshow(y+"2", imgShow)





# cv2.imshow("Image KP1", imgKp1)
cv2.imshow("Image", imgq)
cv2.waitKey()
cv2.destroyAllWindows()

['11.jpeg', '22.jpeg']
StudentName : 0

RegistrationNumber : 202203033

Subject : -

class : .-

totalMarks : 
obtainedMarks : 23

StudentName : 
RegistrationNumber : 202202033

Subject : 
class : 8.-

totalMarks : 2

obtainedMarks : 238



In [3]:
cv2.destroyAllWindows()

## GUI

In [15]:
import tkinter as tk

def open_camera_config():
    # Code to open camera configuration window
    pass

root = tk.Tk()
root.title("Examination Sheet Automation")

# Set the window size to the maximum available on the desktop
root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))

# Create the toolbar frame
toolbar = tk.Frame(root, bd=1, relief=tk.RAISED)

# Create the file menu and add it to the toolbar
file_menu = tk.Menu(toolbar, tearoff=False)
file_menu.add_command(label="Open", command=lambda: print("Open"))
file_menu.add_command(label="Save", command=lambda: print("Save"))
file_menu.add_separator()
file_menu.add_command(label="Camera Configuration", command=open_camera_config)
toolbar_file = tk.Menu(toolbar, tearoff=False)
toolbar_file.add_cascade(label="File", menu=file_menu)
toolbar.add_cascade(label="File", menu=toolbar_file)

# Add the toolbar to the window
root.config(menu=toolbar)


root.mainloop()


AttributeError: 'Frame' object has no attribute 'add_cascade'

In [23]:
import tkinter as tk
import cv2

def open_camera_config():
    # Code to open camera configuration window
    pass

def get_camera_list():
    # Get the list of available cameras
    camera_list = []
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            camera_list.append(f"Camera {i}")
            cap.release()
    return camera_list

root = tk.Tk()
root.title("Examination Sheet Automation")

# Set the window size to the maximum available on the desktop
root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))

# Create the menu bar
menu_bar = tk.Menu(root)

# Create the File menu and add it to the menu bar
file_menu = tk.Menu(menu_bar, tearoff=False)
file_menu.add_command(label="Open", command=lambda: print("Open"))
file_menu.add_command(label="Save", command=lambda: print("Save"))
menu_bar.add_cascade(label="File", menu=file_menu)

# Create the Edit menu and add it to the menu bar
edit_menu = tk.Menu(menu_bar, tearoff=False)
camera_list = get_camera_list()
camera_menu = tk.Menu(edit_menu, tearoff=False)
for camera in camera_list:
    camera_menu.add_command(label=camera, command=lambda: print(camera))
edit_menu.add_cascade(label="Select Camera", menu=camera_menu)
menu_bar.add_cascade(label="Camera Configuration", menu=edit_menu)

# Set the menu bar as the main menu of the window
root.config(menu=menu_bar)

root.mainloop()


Camera 1
Camera 1
Open


In [26]:
import tkinter as tk
import cv2

def open_camera_config():
    # Code to open camera configuration window
    pass

def get_camera_list():
    # Get the list of available cameras
    camera_list = {}
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            camera_list[f"Camera {i}"] = False
            cap.release()
    return camera_list

def toggle_camera(camera_index):
    global camera_list
    camera_label = list(camera_list.keys())[camera_index]
    camera_list[camera_label] = not camera_list[camera_label]
    camera_menu.entryconfig(camera_index, label=f"{camera_label} ✓" if camera_list[camera_label] else camera_label)

root = tk.Tk()
root.title("Examination Sheet Automation")

# Set the window size to the maximum available on the desktop
root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))

# Create the menu bar
menu_bar = tk.Menu(root)

# Create the File menu and add it to the menu bar
file_menu = tk.Menu(menu_bar, tearoff=False)
file_menu.add_command(label="Open", command=lambda: print("Open"))
file_menu.add_command(label="Save", command=lambda: print("Save"))
menu_bar.add_cascade(label="File", menu=file_menu)

# Create the Edit menu and add it to the menu bar
edit_menu = tk.Menu(menu_bar, tearoff=False)
camera_list = get_camera_list()
camera_menu = tk.Menu(edit_menu, tearoff=False)
for i, camera in enumerate(camera_list):
    camera_menu.add_command(label=camera, command=lambda camera_index=i: toggle_camera(camera_index))
edit_menu.add_cascade(label="Select Camera", menu=camera_menu)
menu_bar.add_cascade(label="Camera Configuration", menu=edit_menu)

# Set the menu bar as the main menu of the window
root.config(menu=menu_bar)

root.mainloop()


In [2]:
import tkinter as tk
import cv2

def open_camera_config():
    # Code to open camera configuration window
    pass

def get_camera_list():
    # Get the list of available cameras
    camera_list = {}
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            camera_list[f"Camera {i}"] = False
            cap.release()
    return camera_list

def toggle_camera(camera_label):
    # Toggle the selected status of the camera and update the label
    camera_list[camera_label] = not camera_list[camera_label]
    camera_menu.entryconfig(camera_label, label=f"{camera_label} ✓" if camera_list[camera_label] else camera_label)

def open_camera(camera_label):
    # Open the selected camera in a new window
    cap = cv2.VideoCapture(int(camera_label.split()[-1]))
    cv2.namedWindow(camera_label)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow(camera_label, frame)
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

root = tk.Tk()
root.title("Examination Sheet Automation")

# Set the window size to the maximum available on the desktop
root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))

# Create the menu bar
menu_bar = tk.Menu(root)

# Create the File menu and add it to the menu bar
file_menu = tk.Menu(menu_bar, tearoff=False)
file_menu.add_command(label="Open", command=lambda: print("Open"))
file_menu.add_command(label="Save", command=lambda: print("Save"))
menu_bar.add_cascade(label="File", menu=file_menu)

# Create the Edit menu and add it to the menu bar
edit_menu = tk.Menu(menu_bar, tearoff=False)
camera_list = get_camera_list()
camera_menu = tk.Menu(edit_menu, tearoff=False)
for camera in camera_list:
    camera_menu.add_command(label=camera, command=lambda camera=camera: (toggle_camera(camera), open_camera(camera)) if not camera_list[camera] else None)
edit_menu.add_cascade(label="Select Camera", menu=camera_menu)
menu_bar.add_cascade(label="Camera Configuration", menu=edit_menu)

# Set the menu bar as the main menu of the window
root.config(menu=menu_bar)

root.mainloop()


: 

: 